## Flower Classification
In this programming assignment, we will re-visit the Flower Classification task and solve it using deep neural networks, more specifically
Convolutional Neural Networks. Recap that the flower images are divided into five classes: chamomile, tulip, rose, sunflower, dandelion.
For each class there are about 800 photos. Photos are not high resolution, about 320x240 pixels. Photos are not reduced to a single size, they have different proportions!

You are free to use any deep learning frameworks to build, train and evaluate your models, e.g. PyTorch, Tensorflow 2.x(or its high level API Keras), MXNet and etc. You are free to use already-built models from these libraries like ResNet-50, InceptionNet-V2, and etc, and adding customized classification layer on top of it for your own purpoose.

For computational resource, you will be required to use GPU to train your neural network. Luckily, Google Colab provides free GPU option, you can use that, though it's not very powerful(limited computational speed and limited memory). But in general, for relatively small deep networks(e.g, VGG-16) and small batch size, you can get decent baseline results faster. So always try smaller networks first, and try to select appropriate batch size such that your GPU memories don't overload. 

You are free to use pretrained weights from these established models, and apply transfer learning method by either fixing the weight of pretrained weights and only training on the classification head, or use fine-tuning strategy(small learning rate on whole network.) Transfer learning strategy can accelerate your training progress a lot, you might want to use the strategy when you use those larger networks(i.e. ResNet-50, InceptionNet etc).

To help you detect whether your network and training pipelines are built correctly, a common strategy is to train your model on a very small portion of your training set for a few epoches(should be pretty fast), and to check whether the model can be overfitted(achiving very high accuracy) on this small portion. If the model overfits, then it means that your pipeline setup has no problem, then you can proceed to start your whole training task.

Submit your results to Kaggle competition: https://www.kaggle.com/c/ee596-flower-classification-deep-neural-networks/

## Step 0: Import Necessary Modules
Here we use PyTorch as an example, you can use whatever libraries we want. Torch is the basic backend for the pytorch framework, and torchvision provides you with image proocessing methods before applied to deep neural networks.

In [2]:
import torch
import torchvision
import torchvision.transforms as transform
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from sklearn.model_selection import train_test_split

import torch.nn as nn
import torchvision.models as models
from torchvision.utils import make_grid
import torch.nn.functional as F

## Step1: Data Preprocessing and Loading
Here in order to provide the image tensor that can be input to your network, the following basic steps you need to
follow. 1. Resize all images to the same resolution which is required by your neural network. 2. Normalize your pixel values to small range, say -1 to 1. 

This 2 steps are the most basic steps in feeding images to a CNN, but you can add more on top of it, which is called image augmentation. Image augmentation is a process where you randomly choose a particular transform or perturbation on your original image(Flip, rotate, crop, add noise etc) such that you are creating more samples for your training, which potentially increase the robustness of your model.

The easiest way for you to start with preprocessing is using the built-in transforms in the torchvision module.

In [13]:
simple_transform = transform.Compose([
                                 transform.Resize((220, 220)),
                                 transform.ToTensor(), 
                                 transform.Normalize((0.4124234616756439, 0.3674212694168091, 0.2578217089176178), 
                                                     (0.3268945515155792, 0.29282665252685547, 0.29053378105163574))])

Before Loading, you will create a Dataset class to iterate through your image data. Here we use ImageFolder dataset
which is sub-class of Dataset class for you to iterate through images. Check out torch.utils.Dataset to see how you
can define your own dataset to iterate through images.

In [14]:
path = "./flower_train/"
original = ImageFolder(path, transform=simple_transform)
train, val = train_test_split(original, test_size=0.2, shuffle=True, random_state=43)

In [15]:
for i in range(len(train)):
    img, lab = train.__getitem__(i)
    print(img.shape, lab)

torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 

torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 

torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 

torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 

torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 

torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 

torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 

torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 

torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 3
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 2
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 1
torch.Size([3, 220, 220]) 4
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 220]) 0
torch.Size([3, 220, 

Create the dataloader such that your data can be fed into the network in batches

In [ ]:
# Number of workers enable parallel loading
bs = 16 # Batch size
loaders = {
    'train': DataLoader(train, batch_size=bs, num_workers=4, pin_memory=True),
    'val': DataLoader(val, batch_size=bs, num_workers=4, pin_memory=True),
}

dataset_sizes = {
    'train': len(train),
    'val': len(val)
}

## Step 2: Define the evaluation metrics(Accuracy)

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1) 
    return torch.tensor(torch.sum(preds == labels).item() / len(preds)), preds

##  Step 3: Training Pipeline - Train and validation each epoches, plot curves etc.

In [10]:
def train(epochs, model):
    """
    
    """
    ## Step 1: Send model to GPU device if GPU available, creating Loss function.
    print('Creating a model...')
    device= torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    
    ## Step 2: Create Optimizer, here we use pre-built and pretrained models like VGG-19, we use transfer
    ## learning so only train the classifier part or fully connected part while keeping the backbone of 
    ## the network weight fixed. Adam is the gradient descent algoritm we want to use as default.
    optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)

    since = time.time()
    best_model = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    
    ## Loop for train and val for every epoch
    for epoch in range(epochs):
        for phase in ['train', 'val']:
            
            ## Mode activation, before training network, you must call model.train(), before validation, you must call model.eval()
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0.0
            
            ## Loading data samples from your data loaders and feeding them into the network for training
            for inputs, labels in loaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                ## Clear the gradient for last batch
                optimizer.zero_grad()

                ## Only record gradient when training, validation only forward pass the tensor, no gradient information required, 
                ## so fewer memory need to be allocated.
                with torch.set_grad_enabled(phase=='train'):
                    outp = model(inputs)
                    _, pred = torch.max(outp, 1)
                    loss = criterion(outp, labels)

                    if phase == 'train':
                        ## Back propagation and gradient update
                        loss.backward()
                        optimizer.step()
                ## Sum up the loss and compute accuracy.
                running_loss += loss.item()*inputs.size(0)
                running_corrects += torch.sum(pred == labels.data)

            if phase == 'train':
                acc = 100. * running_corrects.double() / dataset_sizes[phase]
                scheduler.step(acc)
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double()/dataset_sizes[phase]
            losses[phase].append(epoch_loss)
            accuracies[phase].append(epoch_acc)
      
            if phase == 'train':
                print('Epoch: {}/{}'.format(epoch+1, epochs))
                
                
            print('{} - loss:{}, accuracy{}'.format(phase, epoch_loss, epoch_acc))
            lr.append(scheduler._last_lr)
        
            if phase == 'val':
                print('Time: {}m {}s'.format((time.time()- since)//60, (time.time()- since)%60))
                print('=='*31)
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model = copy.deepcopy(model.state_dict())
                
    time_elapsed = time.time() - since
    print('CLASSIFIER TRAINING TIME {}m {}s'.format(time_elapsed//60, time_elapsed%60))
    print('=='*31)
    model.load_state_dict(best_model)
    return model

## Step 4: Initiate Your model

In [9]:
vgg19_bn = torchvision.models.vgg19_bn(pretrained=True)
for param in vgg19_bn.parameters():
    param.grad_requires = False

vgg19_bn.classifier[6] = nn.Linear(4096, len(original.classes), bias=True)

Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /Users/stlp/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

NameError: name 'original' is not defined

## Step 5: Start Training

In [ ]:
epochs = 10
train(epochs=epochs, model=vgg19_bn)

## Step 6: Evaluation on your test set, show the classification accuracy, confusion matrix, do some simple analysis on which flower categories are easily misclassified.